<a href="https://colab.research.google.com/github/maancham/Research-Materials/blob/main/Movielens_trimming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install wget
!pip install csv2tsv

In [2]:
import pandas as pd
import numpy as np
import re
from scipy import stats
import wget

pd.options.mode.chained_assignment = None

from google.colab import drive
# drive.mount('/content/drive')

In [25]:
%%capture
wget.download('https://files.grouplens.org/datasets/movielens/ml-25m.zip')
!unzip ml-25m.zip

df = pd.read_csv('/content/ml-25m/ratings.csv')
movie_df = pd.read_csv('/content/ml-25m/movies.csv')

In [28]:
def extract_year(title):
  if(len(title) > 7):
    year = title[-7:]
    return year[year.find("(")+1:year.find(")")]
  else:
    return None
  
def filter_data(min_year, movie_df, df, n_core_movies = 10, n_core_users = 20):
  movie_df['year'] = movie_df['title'].apply(extract_year)
  movie_df['year'] = pd.to_numeric(movie_df['year'], errors='coerce')
  movie_df = movie_df[movie_df['year'].notna()]
  movie_df['year'] = movie_df.loc[:, 'year'].astype(int)

  movie_df['title'] = movie_df['title'].str.replace(r'\(.*$', '', regex=True)
  movie_df['genres'] = movie_df['genres'].str.replace(r'|', ' ', regex=True)

  ### Filtering based on released year
  movie_df = movie_df[movie_df['year'] >= min_year]

  ### Filtering based on genre availability
  movie_df = movie_df[movie_df['genres'] != '(no genres listed)']


  merged_df = pd.merge(df, movie_df, on='movieId')

  by_movie = merged_df.groupby(by = 'movieId').count()
  unknown_movies = by_movie[by_movie['userId'] < n_core_movies].index.to_list()
  movie_df = movie_df[~movie_df['movieId'].isin(unknown_movies)]

  df = df[df['movieId'].isin(movie_df['movieId'].to_list())]
  by_user_rating = df.groupby(by = 'userId').count().rating
  low_userIds = by_user_rating[by_user_rating < n_core_users].index.to_list()
  df = df[~df['userId'].isin(low_userIds)]
  df.reset_index(inplace=True, drop=True)

  return movie_df, df

In [29]:
movie_df, df = filter_data(1975, movie_df, df)

print("Number of interactions: ", len(df))
print("Number of items: ", len(movie_df))

Number of interactions:  22470243
Number of items:  21133


### Dataset extension part:

In [ ]:
new_user_df = pd.read_csv('/content/drive/MyDrive/research/new user ratings/new_user_houmch.csv', 
                          index_col=0)
new_ratings = new_user_df[['userId', 'movieId', 'rating', 'timestamp']]
new_df = pd.concat([df, new_ratings])
assert(len(df) + len(new_user_df) == len(new_df))

col_names = ['user_id:token','item_id:token','rating:float', 'timestamp:float']
new_df.columns = col_names


file_name = 'ml-25m.inter'
path = 'drive/MyDrive/research/atomic files/' + file_name
new_df.to_csv(path, sep="\t", index=False)

In [ ]:
item_cols = ['movieId', 'title', 'year', 'genres']
movie_df = movie_df[item_cols]

icol_names = ['item_id:token', 'movie_title:token_seq', 'release_year:token', 'class:token_seq']
movie_df.columns = icol_names

file_name = 'ml-25m.item'
path = 'drive/MyDrive/research/atomic files/' + file_name
movie_df.to_csv(path, index=False)